In [1]:
!pip install SPARQLWrapper pandas

import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
from IPython.display import display, HTML
import time

# Configure display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 569.0/569.0 kB 11.4 MB/s eta 0:00:00


In [2]:
# COMMON PREFIXES FOR DBpedia FILM QUERIES
PREFIXES = """
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
"""
print("DBpedia Film SPARQL Prefixes Defined:")
print(PREFIXES)

DBpedia Film SPARQL Prefixes Defined:

PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>



In [3]:
class DBpediaFilmExecutor:
    def __init__(self):
        self.endpoint = "https://dbpedia.org/sparql"
        self.sparql = SPARQLWrapper(self.endpoint)
        self.sparql.setTimeout(120)  # Increase timeout for complex queries

    def execute_query(self, query, description, delay=2):
        """Execute SPARQL query and display results"""

        display(HTML(f"<h3>{description}</h3>"))
        display(HTML(f"<h4> Query:</h4>"))

        # Combine prefixes with query
        full_query = PREFIXES + query
        display(HTML(f"<pre><code>{full_query}</code></pre>"))

        display(HTML(f"<h4> Results:</h4>"))

        try:
            self.sparql.setQuery(full_query)
            self.sparql.setReturnFormat(JSON)

            # Add delay to be respectful to the endpoint
            time.sleep(delay)

            results = self.sparql.query().convert()

            if 'boolean' in results:
                display(HTML(f"<p>ASK Result: <b>{results['boolean']}</b></p>"))
            elif 'results' in results and results['results']['bindings']:
                df = self._format_dataframe(results)
                display(HTML(f"<p>Found {len(df)} results</p>"))
                display(df)
            else:
                display(HTML("<p>No results found</p>"))
                display(HTML("<p><i>Try modifying the query parameters</i></p>"))

        except Exception as e:
            display(HTML(f"<p>Query execution error: {str(e)}</p>"))
            display(HTML("<p><i>This might be due to endpoint load or query complexity</i></p>"))

    def _format_dataframe(self, results):
        """Convert SPARQL results to pandas DataFrame"""
        columns = results['head']['vars']
        data = []

        for result in results['results']['bindings']:
            row = {}
            for column in columns:
                if column in result:
                    # Extract just the value for display
                    value = result[column]['value']
                    # Shorten long URIs for better display
                    if 'dbpedia.org' in value and '/' in value:
                        value = value.split('/')[-1]
                    row[column] = value
                else:
                    row[column] = None
            data.append(row)

        return pd.DataFrame(data)

# Initialize executor
executor = DBpediaFilmExecutor()

print("DBpedia Film SPARQL Executor Ready!")

DBpedia Film SPARQL Executor Ready!


In [4]:
query_1 = """
SELECT ?film ?title ?releaseDate ?director ?runtime
WHERE {
    ?film rdf:type dbo:Film ;
          rdfs:label ?title ;
          dbo:releaseDate ?releaseDate .

    # Optional director information
    OPTIONAL {
        ?film dbo:director ?director .
        ?director foaf:name ?directorName .
    }

    # Optional runtime information
    OPTIONAL {
        ?film dbo:runtime ?runtime .
    }

    FILTER (YEAR(?releaseDate) >= 2010 && YEAR(?releaseDate) <= 2015)
    FILTER (LANG(?title) = 'en')
}
ORDER BY ?releaseDate
LIMIT 20
"""

executor.execute_query(query_1, "Query 1: Films from 2010-2015 with Basic Information")

,film,title,releaseDate,director,runtime
0,Natarang,Natarang,2010-01-01,Ravi_Jadhav,7200.0
1,Ryōmaden,Ryōmaden,2010-01-03,Keishi_Ōtomo,4500.0
2,Ryōmaden,Ryōmaden,2010-01-03,Keishi_Ōtomo,2700.0
3,Tough_Nights_of_Club_Indigo,Tough Nights of Club Indigo,2010-01-05,None,None
4,Trauma_(Canadian_TV_series),Trauma (Canadian TV series),2010-01-05,None,None
5,The_Pharaoh_Who_Conquered_the_Sea,The Pharaoh Who Conquered the Sea,2010-01-06,None,None
6,Black_Ransom,Black Ransom,2010-01-07,Wong_Jing,5580.0
7,Shoot_the_Hero!,Shoot the Hero!,2010-01-08,None,4800.0
8,Ploddy_the_Police_Car_Makes_a_Splash,Ploddy the Police Car Makes a Splash,2010-01-08,Rasmus_A._Sivertsen,4440.0
9,Mighty_Uke,Mighty Uke,2010-01-11,None,4560.0


In [5]:
query_2 = """
SELECT ?decade
       (COUNT(?film) AS ?filmCount)
       (AVG(?runtime) AS ?avgRuntime)
       (MIN(YEAR(?releaseDate)) AS ?firstYear)
       (MAX(YEAR(?releaseDate)) AS ?lastYear)
WHERE {
    ?film rdf:type dbo:Film ;
          dbo:releaseDate ?releaseDate .

    # Optional runtime for average calculation
    OPTIONAL {
        ?film dbo:runtime ?runtime .
    }

    # Group by decade
    BIND (FLOOR(YEAR(?releaseDate)/10)*10 AS ?decade)

    FILTER (YEAR(?releaseDate) >= 1980)
}
GROUP BY ?decade
HAVING (COUNT(?film) > 10)
ORDER BY ?decade
"""

executor.execute_query(query_2, "Query 2: Film Statistics by Decade (1980+)")

,decade,filmCount,avgRuntime,firstYear,lastYear
0,1980,1907,6126.796742671009772,1980,1989
1,1990,2285,5976.766944734098019,1990,1999
2,2000,3513,6844.165820178448868,2000,2009
3,2010,2552,6358.278006012024048,2010,2019
4,2020,356,5768.654901960784314,2020,2024


In [6]:
query_3 = """
SELECT ?film ?title ?releaseDate ?genre ?country ?language ?budget
WHERE {
    ?film rdf:type dbo:Film ;
          rdfs:label ?title ;
          dbo:releaseDate ?releaseDate .

    # Optional genre information
    OPTIONAL {
        ?film dbo:genre ?genre .
        ?genre rdfs:label ?genreLabel .
        FILTER (LANG(?genreLabel) = 'en')
    }

    # Optional country information
    OPTIONAL {
        ?film dbo:country ?country .
        ?country rdfs:label ?countryLabel .
        FILTER (LANG(?countryLabel) = 'en')
    }

    # Optional language information
    OPTIONAL {
        ?film dbo:language ?language .
        ?language rdfs:label ?languageLabel .
        FILTER (LANG(?languageLabel) = 'en')
    }

    # Optional budget information
    OPTIONAL {
        ?film dbo:budget ?budget .
    }

    FILTER (YEAR(?releaseDate) >= 2020)
    FILTER (LANG(?title) = 'en')
}
ORDER BY DESC(?releaseDate)
LIMIT 25
"""

executor.execute_query(query_3, "Query 3: Recent Films (2020+) with Complete Metadata")

,film,title,releaseDate,genre,country,language,budget
0,100_Years_(film),100 Years (film),2115-11-18,None,None,None,None
1,Force_(film_series),Force (film series),2024-08-15,None,None,None,None
2,The_Nun_2,The Nun 2,2023-09-08,None,None,None,None
3,Gadar_2,Gadar 2,2023-08-15,None,None,None,None
4,Carry_On_Jatta_(film_series),Carry On Jatta (film series),2023-06-29,None,Canada,None,13.5
5,Carry_On_Jatta_(film_series),Carry On Jatta (film series),2023-06-29,None,India,None,13.5
6,Jodi_(2022_film),Jodi (2022 film),2023-05-05,None,None,Punjabi_language,None
7,Money_Back_Guarantee_(2023_film),Money Back Guarantee (2023 film),2023-04-21,None,None,Urdu,None
8,Selfiee_(2023_film),Selfiee (2023 film),2023-02-24,None,None,None,None
9,Vaathi,Vaathi,2023-02-17,None,None,None,None


In [7]:
query_4 = """
SELECT ?directorName
       (COUNT(DISTINCT ?film) AS ?totalFilms)
       (MIN(YEAR(?releaseDate)) AS ?firstFilmYear)
       (MAX(YEAR(?releaseDate)) AS ?latestFilmYear)
       (MAX(YEAR(?releaseDate)) - MIN(YEAR(?releaseDate)) AS ?careerSpan)
       (SAMPLE(?recentFilmTitle) AS ?recentFilm)
WHERE {
    ?film rdf:type dbo:Film ;
          dbo:director ?director ;
          dbo:releaseDate ?releaseDate ;
          rdfs:label ?recentFilmTitle .

    ?director foaf:name ?directorName .

    FILTER (YEAR(?releaseDate) >= 1990)
    FILTER (LANG(?recentFilmTitle) = 'en')
}
GROUP BY ?directorName
HAVING (COUNT(DISTINCT ?film) >= 5)
ORDER BY DESC(?totalFilms) DESC(?careerSpan)
LIMIT 15
"""

executor.execute_query(query_4, "Query 4: Director Filmography Analysis")

,directorName,totalFilms,firstFilmYear,latestFilmYear,careerSpan,recentFilm
0,,26,1990,2020,30,School Wars: Hero
1,Chuck Jones,19,1994,2020,26,Chariots of Fur
2,Friz Freleng,18,2003,2020,17,Looney Tunes Golden Collection: Volume 1
3,Takashi Miike,16,1991,2006,15,Dead or Alive 2: Birds
4,Udayakantha Warnasuriya,11,1998,2019,21,President Super Star
5,Ram Gopal Varma,10,1990,2017,27,Sarkar (film series)
6,Herman Yau,10,1992,2019,27,Best of the Best (1992 film)
7,P Vasu,10,1992,2017,25,Hrudayavantha
8,Joseph Sargent,10,1990,2008,18,Caroline?
9,Arthur Davis,10,2003,2014,11,Looney Tunes Golden Collection: Volume 1


In [11]:
query_5 = """
SELECT ?season ?genreLabel
       (COUNT(?film) AS ?filmCount)
       (AVG(?runtime) AS ?avgRuntime)
WHERE {
    ?film rdf:type dbo:Film ;
          dbo:releaseDate ?releaseDate ;
          dbo:genre ?genre .

    ?genre rdfs:label ?genreLabel .

    # Optional runtime for average calculation
    OPTIONAL {
        ?film dbo:runtime ?runtime .
    }

    # Determine season based on release month
    BIND (
        IF(MONTH(?releaseDate) IN (12, 1, 2), "Winter",
        IF(MONTH(?releaseDate) IN (3, 4, 5), "Spring",
        IF(MONTH(?releaseDate) IN (6, 7, 8), "Summer", "Autumn")))
        AS ?season
    )

    FILTER (YEAR(?releaseDate) >= 2000)
    FILTER (LANG(?genreLabel) = 'en')
}
GROUP BY ?season ?genreLabel
HAVING (COUNT(?film) >= 5)
ORDER BY ?season DESC(?filmCount)
LIMIT 30
"""

executor.execute_query(query_5, "Query 5: Seasonal Release Patterns by Genre")

,season,genreLabel,filmCount,avgRuntime
0,Autumn,Sitcom,15,1242.857142857142857
1,Autumn,Drama,13,4800
2,Autumn,Romance film,12,2501.25
3,Autumn,Reality television,11,2716.363636363636364
4,Autumn,Telenovela,11,40
5,Autumn,Comedy,10,1911.428571428571429
6,Autumn,Documentary,9,2446.666666666666667
7,Autumn,Tokusatsu,8,-171.428571428571429
8,Autumn,Superhero fiction,8,-171.428571428571429
9,Autumn,Satire,6,3170


In [12]:
query_advanced = """
SELECT ?seriesName
       (COUNT(?film) AS ?filmsInSeries)
       (MIN(YEAR(?releaseDate)) AS ?firstRelease)
       (MAX(YEAR(?releaseDate)) AS ?latestRelease)
       (GROUP_CONCAT(DISTINCT ?filmTitle; SEPARATOR=", ") AS ?filmTitles)
WHERE {
    ?film rdf:type dbo:Film ;
          rdfs:label ?filmTitle ;
          dbo:releaseDate ?releaseDate ;
          dct:subject ?category .

    ?category rdfs:label ?seriesName .

    FILTER (LANG(?filmTitle) = 'en')
    FILTER (LANG(?seriesName) = 'en')
    FILTER (CONTAINS(LCASE(?seriesName), "film series") ||
            CONTAINS(LCASE(?seriesName), "films") ||
            CONTAINS(LCASE(?seriesName), "franchise"))
    FILTER (YEAR(?releaseDate) >= 1990)
}
GROUP BY ?seriesName
HAVING (COUNT(?film) >= 3)
ORDER BY DESC(?filmsInSeries) DESC(?latestRelease)
LIMIT 20
"""

executor.execute_query(query_advanced, "Advanced: Film Series and Franchise Analysis")

,seriesName,filmsInSeries,firstRelease,latestRelease,filmTitles
0,2000s English-language films,944,1998,2017,"$9.99, 'Twas the Night, 101 Rent Boys, 101 Rey..."
1,1990s English-language films,792,1990,2012,"(Blooper) Bunny, ...First Do No Harm, 1991: Th..."
2,1990s American films,588,1990,2012,"...First Do No Harm, 12 Angry Men (1997 film),..."
3,2000s American films,577,2000,2016,"'Twas the Night, 10,000 Black Men Named George..."
4,2010s English-language films,479,2000,2020,"10 Hours of Walking in NYC as a Woman, 16-Love..."
5,2010s American films,306,2010,2020,"16-Love, 3 Day Weekend, 30 Years from Here, 7 ..."
6,2006 films,298,2006,2016,"9/11: The Twin Towers, A Christmas Carol (2006..."
7,2001 films,286,2000,2011,"'Twas the Night, A Crack in the Floor, A Funny..."
8,2004 films,278,2002,2017,"10.5 (miniseries), A Christmas Carol (2004 fil..."
9,2007 films,272,1999,2015,"2 Girls 1 Cup, A Family Lost, A Love Story (20..."
